### Variational Autoencoder

In [4]:
### importing modules that are needed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn 
import torch.nn.functional as f 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split



##### **Loading the data**

In [5]:
data_f = pd.read_csv(\
    '~/ml_J1-J2_supervised/all_phase/af/augumented_dataL24.csv',index_col=[0])

X_train,X_test,y_train,y_test = train_test_split(data_f.iloc[:,:-1],data_f.iloc[:,-1:], \
                    random_state=42,test_size=0.2,stratify=data_f.iloc[:,-1:])

data_train = pd.concat([X_train,y_train],axis=1)
data_test = pd.concat([X_test,y_test],axis=1)



**Set the device**

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##### Class definition that is used to load the dataset.

In [12]:
### class to load the data

class LoadData(Dataset):
    def __init__(self,data,L,device=device):
        self.L = L
        self.x_data = torch.tensor(data.iloc[:,:-1].values,dtype=torch.float32).to(device=device)
        self.y_data = torch.tensor(data.iloc[:,-1:].values,dtype=torch.long).to(device=device)

    ### length of the dataset
    ### function one has to use if you want to define a custom dataset class
    def __len__(self):
        return len(self.y_data)

    
    ## get the image and label
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        image = self.x_data[idx,:]
        label = self.y_data[idx]

        return {'data':image,'label':label}

##### **Model definition for the variational autoencoder**

In [13]:
### definition of the network
class Vae(nn.Module):
    
    def __init__(self,input_size,latent_dim):
        super(Vae,self).__init__()

        ### encoder network
        self.encoder1 = nn.Linear(in_features=input_size,out_features=256)
        self.encoder2 = nn.Linear(in_features=256,out_features=128)
        self.encoder3 = nn.Linear(in_features=128,out_features=64)
        self.encoder4 = nn.Linear(in_features=64,out_features=32)

        self.mu = nn.Linear(in_features=32,out_features=latent_dim)
        self.var = nn.Linear(in_features=32,out_features=latent_dim)

        self.decoder1 = nn.Linear(in_features=latent_dim,out_features=32)
        self.decoder2 = nn.Linear(in_features=32, out_features=64)
        self.decoder3 = nn.Linear(in_features=64, out_features=128)
        self.decoder4 = nn.Linear(in_features=128, out_features=256)

    def forward(self,x):
        x = self.encoder1(x)
        x = self.encoder2(x)
        x = self.encoder3(x)
        x = self.encoder4(x)
        
        x1 = self.mu(x)
        x2 = self.var(x)

        z = torch.distributions.normal(0,1).rsample() * x2 + x1
        x = self.decoder1(z)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)

        return x


IndentationError: unexpected indent (<ipython-input-13-7ca6199ef852>, line 8)

In [18]:
torch.distributions.Normal(0,1).rsample()   

tensor(-0.6672)